In [403]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os.path
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [404]:
parent_url = 'https://www.prokabaddi.com'
teams_rel_loc= 'teams'
players_rel_loc = 'players' 
teams = ['bwarriors','bbulls']
team_loc_dict = {"bwarriors":"bengal-warriors-profile-4",
                "bbulls" : "bengaluru-bulls-profile-1",
              "ddelhi": "dabang-delhi-kc-profile-2",
                "gujgiants": "gujarat-fortunegiants-profile-31",
                "harsteels":"haryana-steelers-profile-28",
                "jppinkpather":"jaipur-pink-panthers-profile-3",
                "patpirates":"patna-pirates-profile-6",
                "punpaltan":"puneri-paltan-profile-7",
                "tamthaliva":"tamil-thalaivas-profile-29",
                "teltitans":"telugu-titans-profile-8",
                "umumba":"u-mumba-profile-5",
                "upyoddha":"up-yoddha-profile-30"}
base_op_dir = './data/'


In [405]:
class utils:
    def make_url(base_url, *rel_locs):
        final_url = base_url
        for loc in rel_locs:
            final_url = final_url + '/' + loc
        return final_url

In [406]:
def write_to_file(filenameprefix, player_meta_data, list_of_statvals, header):
    filename  = filenameprefix+'.csv'
    filepath = '/'.join([base_op_dir,filename])
    print("writing to :"+ filepath)
    list_of_statvals.extend(player_meta_data.values())
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a') as season_file:
        wr = csv.writer(season_file, quoting=csv.QUOTE_ALL)
        if not file_exists:
            wr.writerow(header)
        wr.writerow(list_of_statvals)

In [407]:
def init_stats(browser):
    browser.get(player_url)
    time.sleep(3)
    try:
        stats_page = browser.find_element_by_xpath('//div[@data-tabtype="stats"]')
        stats_page.click()
        print('stats page found... clicking')
    except:
        print('stats not found..do nothing')
        

In [408]:
def get_stats_for_player(browser):
    right_header_section = soup.find('div', class_='si-right').find("div", class_='si-tbl-header')
    col_name_divs = right_header_section.findAll('div',class_='si-data-block')
    col_names = []
    row_names = []
    for col in col_name_divs:
        col_names.append(col.text)
    col_names = [name.replace(' ','') for name in col_names]
    
    prev_section = right_header_section
    stats_dict = dict()
    while 4<5:
        right_data_section = prev_section.findNext("div", class_='si-tbl-wraper')
        if right_data_section:
            all_values = right_data_section.findAll('div', class_='si-tbl-data')
            i=0
            for col_val in all_values:
                val_list=[]
                all_vals_for_column = col_val.findAll('div', class_='si-data-block')
                for val in all_vals_for_column:
                    val_list.append(val.get_text().strip())
                if col_names[i] not in stats_dict:
                    stats_dict['%s' %(col_names[i])]=val_list
                else:
                    stats_dict['%s' %(col_names[i])].extend(val_list)
                i=i+1
        else:
            break
        prev_section = right_data_section
        
    row_name_divs = soup.find('div', class_='si-left').findAll("div", class_='si-tbl-wraper')
    row_names=[]
    for name_div in row_name_divs:
        temp = name_div.findAll("div", class_='si-data-block')
        for row_div in temp:
            row_names.append(row_div.find(text=True, recursive=False))
    row_names.pop(0)    
    return(col_names, row_names, stats_dict)

In [409]:
def init_browser():
    browser = webdriver.Chrome(ChromeDriverManager().install())
    return browser


In [410]:
def get_all_players(browser, team_url):
    browser.get(team_url)
    time.sleep(3)
    soup = BeautifulSoup(browser.page_source, "html.parser")
    name_list = soup.find('div', 'si-page-plyrWrp').findAll("a", {"class": "si-plyrBox"})
    all_player_urls=[]
    for name_tag in name_list:
        all_player_urls.append(utils.make_url(parent_url,name_tag['href']))
    return all_player_urls

In [413]:
browser = init_browser()
headers = []
for team in team_loc_dict.keys():
    print("Team:"+ team)
    team_players_url = utils.make_url(parent_url, 'teams', team_loc_dict[team],'players')
    all_player_urls = get_all_players(browser, team_players_url)
    for player_url in all_player_urls:
        player_meta_data={}
        init_stats(browser)
        soup = BeautifulSoup(browser.page_source, "html.parser")
        player_name = soup.find('span', class_='si-name').get_text()
        player_meta_data['player_name']= player_name
        player_meta_data['team'] = team
        print("Player: "+player_name)
        col_names,row_names,data = get_stats_for_player(browser)
        if len(headers)==0:
            headers = row_names
            headers.extend(player_meta_data.keys())
        for season_val in col_names:
            write_to_file(season_val, player_meta_data, data[season_val], headers)
    time.sleep(5)
browser.close()


Checking for mac64 chromedriver:70.0.3538.97 in cache
Driver found in /Users/aarora7/.wdm/chromedriver/70.0.3538.97/mac64/chromedriver
Team:bwarriors
stats page found... clicking
Player: Maninder Singh
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season2.csv
writing to :./data//Season1.csv
stats page found... clicking
Player: K. Prapanjan
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season4.csv
writing to :./data//Season3.csv
writing to :./data//Season2.csv
stats not found..do nothing
Player: Mohammad Taghi Paein Mahali
writing to :./data//Overall.csv
writing to :./data//Season7.csv
stats not found..do nothing
Player: Rakesh Narwal
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Seaso

stats not found..do nothing
Player: Mohit
writing to :./data//Overall.csv
writing to :./data//Season7.csv
stats not found..do nothing
Player: Vishal Mane
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season4.csv
writing to :./data//Season3.csv
writing to :./data//Season2.csv
writing to :./data//Season1.csv
stats not found..do nothing
Player: Pratik Patil
writing to :./data//Overall.csv
writing to :./data//Season7.csv
stats page found... clicking
Player: Ravinder Pahal
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season4.csv
writing to :./data//Season3.csv
writing to :./data//Season2.csv
writing to :./data//Season1.csv
stats not found..do nothing
Player: Anil Kumar
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season

Player: Kuldeep Singh
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
stats not found..do nothing
Player: Tin Phonchoo
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season4.csv
Team:jppinkpather
stats page found... clicking
Player: Deepak Niwas Hooda
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season4.csv
writing to :./data//Season3.csv
writing to :./data//Season2.csv
writing to :./data//Season1.csv
stats not found..do nothing
Player: Ajinkya Ashok Pawar
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
stats page found... clicking
Player: Deepak Narwal
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :

writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
stats not found..do nothing
Player: Sushant Sail
writing to :./data//Overall.csv
writing to :./data//Season7.csv
stats not found..do nothing
Player: Deepak Yadav
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
stats not found..do nothing
Player: Jadhav Balasaheb Shahaji
writing to :./data//Overall.csv
writing to :./data//Season7.csv
stats not found..do nothing
Player: Hadi Tajik
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season4.csv
writing to :./data//Season3.csv
stats not found..do nothing
Player: Shubham Shinde
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
stats not found..do nothing
Player: Anil
writing to :./data//Overall.cs

writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season4.csv
writing to :./data//Season3.csv
writing to :./data//Season2.csv
stats not found..do nothing
Player: Abhishek Singh
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
stats not found..do nothing
Player: Arjun Deshwal
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
stats page found... clicking
Player: Athul MS
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
writing to :./data//Season4.csv
stats not found..do nothing
Player: Dong Geon Lee
writing to :./data//Overall.csv
writing to :./data//Season7.csv
writing to :./data//Season6.csv
writing to :./data//Season5.csv
stats not found..do nothing
Player: Gaurav Kumar
writing to :./data//Overall.csv
writing to :./data

In [412]:
browser.close()